In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import json
import zipfile
from datetime import datetime

from scipy import stats
#from sklearn.neighbors import LocalOutlierFactor
#clf = LocalOutlierFactor(n_neighbors=5, contamination=0.05)
pd.set_option('display.max_colwidth', 25)
plt.rcParams["font.family"] = "Arial"

In [2]:
import os
import psutil
process = psutil.Process(os.getpid())
print(process.memory_info().rss/1e6)
from sys import getsizeof

157.22496


# 1. gee list to df

In [3]:
if False:
    empty_df = pd.DataFrame([[-99, np.nan]], columns=['group','value']).set_index('group')

    def str2df(mstr, reducer, name):
        if len(mstr) >= 10:
            df = pd.DataFrame.from_records(json.loads(mstr.replace('group', '"group"') \
                                                    .replace(reducer, '"' + reducer + '"').replace('=', ':'))) \
                           .rename({reducer: name}, axis=1).set_index('group').astype('float32')
        else:
            df = empty_df.rename({'value': name}, axis=1)
        return df

    def row2df(row):
        hylak_id       = row['Hylak_id']
        occr_area      = row['occr_area']
        occr_srtm      = row['occr_srtm']
        occr_srtm_nasa = row['occr_srtm_nasa']
        occr_alos2_2   = row['occr_alos2_2']
        occr_alos3_2   = row['occr_alos3_2']
        occr_aster100  = row['occr_aster100']
        occr_aster30   = row['occr_aster30']
        occr_glo30     = row['occr_glo30']

        df0 = str2df(occr_area,      'sum',  'area_m2')      
        df1 = str2df(occr_srtm,      'mean', 'srtm')     
        df2 = str2df(occr_srtm_nasa, 'mean', 'srtm_nasa')     
        df3 = str2df(occr_alos2_2,   'mean', 'alos2_2')     
        df4 = str2df(occr_alos3_2,   'mean', 'alos3_2')     
        df5 = str2df(occr_aster100,  'mean', 'aster100')     
        df6 = str2df(occr_aster30,   'mean', 'aster30')     
        df7 = str2df(occr_glo30,     'mean', 'glo30')     

        df_all = pd.concat([df0, df1, df2, df3, df4, df5, df6, df7], axis=1).reset_index().sort_values('group')
        df_all['hylak_id'] = hylak_id

        return df_all

In [4]:
def str2df(mstr, reducer):
    if len(mstr) >= 10:
        js = json.loads(mstr.replace('group', '"group"').replace(reducer, '"' + reducer + '"').replace('=', ':'))
        dt = {i.get('group'):i.get(reducer) for i in js}
    else:
        dt = {-99: np.nan}
    return dt

def row2df(row):
    hylak_id       = row['Hylak_id']
    occr_area      = row['occr_area']
    occr_srtm      = row['occr_srtm']
    occr_srtm_nasa = row['occr_srtm_nasa']
    occr_alos2_2   = row['occr_alos2_2']
    occr_alos3_2   = row['occr_alos3_2']
    occr_aster100  = row['occr_aster100']
    occr_aster30   = row['occr_aster30']
    occr_glo30     = row['occr_glo30']
    
    dt0 = str2df(occr_area,      'sum')      
    dt1 = str2df(occr_srtm,      'mean')     
    dt2 = str2df(occr_srtm_nasa, 'mean')     
    dt3 = str2df(occr_alos2_2,   'mean')     
    dt4 = str2df(occr_alos3_2,   'mean')     
    dt5 = str2df(occr_aster100,  'mean')     
    dt6 = str2df(occr_aster30,   'mean')     
    dt7 = str2df(occr_glo30,     'mean')     
    
    df_all = pd.DataFrame([dt0,dt1,dt2,dt3,dt4,dt5,dt6,dt7], 
                          index=['area_m2','srtm','srtm_nasa','alos2_2','alos3_2','aster100','aster30','glo30'],
                          dtype='float32') \
               .transpose().reset_index().rename({'index': 'group'}, axis=1)
    df_all['hylak_id'] = hylak_id
        
    return df_all

In [6]:
with zipfile.ZipFile("0_gee_output/hylak_dem_ee_all.zip", "r") as zipf:
    for fname in zipf.namelist():
        #print(fname)
        if '.csv' in fname:
            output = []
            with zipf.open(fname) as myZip:
                mdata = pd.read_csv(myZip)
                print(fname, process.memory_info().rss/1e6, datetime.now())
                
                for index, row in mdata.iterrows():
                    output.append(row2df(row))

            output_df = pd.concat(output)
            output_df = output_df[output_df['group'] != -99]
            output_df.to_parquet("1_occr_dem/" + fname.replace('.csv', '.parquet'), index=False, engine='pyarrow')

display(output_df.head(3))

hylak_dem_0.csv 320.065536 2021-10-03 20:47:27.149976
hylak_dem_100.csv 564.072448 2021-10-03 20:47:58.551340
hylak_dem_101.csv 533.102592 2021-10-03 20:48:27.003290
hylak_dem_102.csv 576.557056 2021-10-03 20:48:55.508025
hylak_dem_103.csv 575.066112 2021-10-03 20:49:26.516246
hylak_dem_104.csv 610.144256 2021-10-03 20:49:56.730372
hylak_dem_105.csv 609.247232 2021-10-03 20:50:28.315169
hylak_dem_106.csv 599.621632 2021-10-03 20:50:56.834204
hylak_dem_107.csv 619.97056 2021-10-03 20:51:26.223648
hylak_dem_108.csv 610.5088 2021-10-03 20:51:57.015198
hylak_dem_109.csv 631.353344 2021-10-03 20:52:28.819490
hylak_dem_10.csv 654.053376 2021-10-03 20:53:00.956341
hylak_dem_110.csv 644.591616 2021-10-03 20:53:33.044824
hylak_dem_111.csv 674.766848 2021-10-03 20:54:04.038247
hylak_dem_112.csv 640.663552 2021-10-03 20:54:33.436464
hylak_dem_113.csv 691.494912 2021-10-03 20:55:05.071230
hylak_dem_114.csv 646.868992 2021-10-03 20:55:36.398744
hylak_dem_115.csv 646.938624 2021-10-03 20:56:06.24800

,group,area_m2,srtm,srtm_nasa,alos2_2,alos3_2,aster100,aster30,glo30,hylak_id
0,2,5445.937988,1.332681,-0.311072,1.531125,1.492908,7.897519,8.781362,1.00000,1389799
1,3,19759.382812,1.003732,-0.490655,1.611780,1.755716,8.626242,8.893746,1.00000,1389799
2,4,9150.261719,1.655110,0.385745,1.964069,2.061518,8.623298,9.094102,1.04121,1389799


# 2. process occr-dem

In [9]:
N = 5
cols = ['srtm_nasa', 'alos3_2', 'aster30','glo30']
def df_process(df_group):
    
    glb_g_min = df_group['group'].min()
    glb_g_max = df_group['group'].max()
    df_all = df_group[df_group['area_m2'] > 900*10]
    
    g_min_sel = 0
    default = 0
    for col in cols:
        
        df_tmp = df_all[['group', col]].copy()
        
        df_tmp = df_tmp.dropna()
        
        ### please note this function is a simplified version
        ### full version will be uploaded once the ms is published
        if len(df_tmp) > 0:
            y_padded = np.pad(df_tmp[col].values, (N//2, N-1-N//2), mode='edge')
            df_tmp[col] = np.convolve(y_padded, np.ones((N,))/N, 'valid')

            g_max, v_max = df_tmp[df_tmp[col] == df_tmp[col].max()].iloc[0][['group', col]]
            g_min, v_min = df_tmp[df_tmp[col] == df_tmp[col].min()].iloc[0][['group', col]]
            
            ### check bottom flat line ###
            vmin1 = v_min + 0.5
            df_tmp_bottom = df_tmp[(df_tmp[col] <= vmin1) & (df_tmp['group'] <= g_min)]
            df_tmp_min = df_tmp[(df_tmp[col] >= vmin1) & (df_tmp['group'] <= g_min)]
            g_min, v_min = df_tmp_min.iloc[-1][['group', col]]

            ### extrapolate the flat line ###
            v_part1 = np.interp(np.arange(glb_g_min, g_min+1, 1), df_tmp_valid['group'], df_tmp_valid[col])
            v_part2 = intercept_adj + res.slope*np.arange(g_min+1, glb_g_max+1, 1)
            col_g = np.arange(glb_g_min, glb_g_max+1, 1)
            col_v = np.concatenate((v_part1, v_part2))
            f_g_max, f_v_max, f_g_min, f_v_min = g_max, v_max, g_min, v_min
            f_col = col
            g_min_sel = g_min
    
    return (final_g, final_v, [f_g_max, f_g_min], [f_v_max, f_v_min], f_col, default)

In [13]:
all_out = []

i = 0
for fname in sorted(glob.glob('1_occr_dem/*.parquet')):

    output_df = pd.read_parquet(fname, columns=['hylak_id','group','area_m2','srtm_nasa','alos3_2',
                                                                   'aster30','glo30'])

    for hylak_id, df_group in output_df.groupby('hylak_id'):

        final_g, final_v, [f_g_max, f_g_min], [f_v_max, f_v_min], f_col, default = df_process(df_group)
        all_out.append([hylak_id, final_g, final_v, f_col, default])
        if i%2000 == 0:
            print(i, process.memory_info().rss/1e6, datetime.now())
        i = i + 1

all_out_df = pd.DataFrame(all_out, columns=['hylak_id', 'final_g', 'final_v', 'source', 'default'])
display(all_out_df)
all_out_df.to_parquet("out_1_occr_dem_ext.parquet", index=False, engine='pyarrow')

0 854.986752 2021-10-03 22:13:02.135102
2000 787.968 2021-10-03 22:13:34.308439
4000 787.968 2021-10-03 22:14:02.446535
6000 787.968 2021-10-03 22:14:30.876821
8000 787.968 2021-10-03 22:14:59.815428
10000 818.192384 2021-10-03 22:15:28.233581
12000 818.192384 2021-10-03 22:16:00.551830
14000 818.192384 2021-10-03 22:16:28.707379
16000 818.192384 2021-10-03 22:16:57.133503
18000 818.192384 2021-10-03 22:17:25.610383
20000 828.411904 2021-10-03 22:17:53.347359
22000 828.411904 2021-10-03 22:18:21.298180
24000 828.411904 2021-10-03 22:18:44.981886
26000 828.411904 2021-10-03 22:19:11.357639
28000 828.411904 2021-10-03 22:19:37.754044
30000 824.430592 2021-10-03 22:20:06.376575
32000 824.430592 2021-10-03 22:20:34.263295
34000 824.430592 2021-10-03 22:21:01.150676
36000 824.430592 2021-10-03 22:21:28.030400
38000 824.430592 2021-10-03 22:21:54.848486
40000 823.97184 2021-10-03 22:22:22.444005
42000 823.97184 2021-10-03 22:22:50.454963
44000 823.97184 2021-10-03 22:23:17.888788
46000 823.9

366000 1269.16608 2021-10-03 23:32:40.384089
368000 1269.952512 2021-10-03 23:33:06.972845
370000 1270.4768 2021-10-03 23:33:30.642197
372000 1271.263232 2021-10-03 23:33:53.623333
374000 1271.78752 2021-10-03 23:34:21.515127
376000 1271.754752 2021-10-03 23:34:49.671197
378000 1272.27904 2021-10-03 23:35:15.063253
380000 1273.065472 2021-10-03 23:35:39.470166
382000 1273.58976 2021-10-03 23:36:05.304230
384000 1274.376192 2021-10-03 23:36:32.955032
386000 1303.01952 2021-10-03 23:37:00.778895
388000 1303.805952 2021-10-03 23:37:26.685009
390000 1304.33024 2021-10-03 23:37:51.146850
392000 1305.116672 2021-10-03 23:38:17.131038
394000 1305.64096 2021-10-03 23:38:45.374674
396000 1318.346752 2021-10-03 23:39:16.736622
398000 1318.87104 2021-10-03 23:39:43.581388
400000 1319.657472 2021-10-03 23:40:11.419193
402000 1320.18176 2021-10-03 23:40:40.714877
404000 1320.968192 2021-10-03 23:41:07.355938
406000 1326.936064 2021-10-03 23:41:38.750305
408000 1327.722496 2021-10-03 23:42:06.940981

726000 1889.460224 2021-10-04 00:54:35.273983
728000 1890.246656 2021-10-04 00:55:03.521243
730000 1896.05888 2021-10-04 00:55:32.897656
732000 1896.845312 2021-10-04 00:56:00.728420
734000 1897.3696 2021-10-04 00:56:28.200349
736000 1898.156032 2021-10-04 00:56:54.852708
738000 1898.68032 2021-10-04 00:57:19.944776
740000 1927.667712 2021-10-04 00:57:47.689195
742000 1928.192 2021-10-04 00:58:16.097039
744000 1928.978432 2021-10-04 00:58:43.587786
746000 1929.50272 2021-10-04 00:59:10.914162
748000 1930.289152 2021-10-04 00:59:37.856658
750000 1935.958016 2021-10-04 01:00:05.919071
752000 1936.744448 2021-10-04 01:00:32.700495
754000 1937.268736 2021-10-04 01:00:59.125130
756000 1938.055168 2021-10-04 01:01:24.094394
758000 1938.579456 2021-10-04 01:01:46.400612
760000 1968.5376 2021-10-04 01:02:14.573756
762000 1969.061888 2021-10-04 01:02:45.417252
764000 1969.84832 2021-10-04 01:03:14.985031
766000 1970.372608 2021-10-04 01:03:45.115491
768000 1971.15904 2021-10-04 01:04:14.070872


1084000 2541.682688 2021-10-04 02:12:15.662742
1086000 2542.206976 2021-10-04 02:12:39.649799
1088000 2540.969984 2021-10-04 02:13:09.140704
1090000 2541.494272 2021-10-04 02:13:38.127331
1092000 2542.280704 2021-10-04 02:14:05.080685
1094000 2542.804992 2021-10-04 02:14:32.464988
1096000 2543.591424 2021-10-04 02:15:02.030611
1098000 2545.958912 2021-10-04 02:15:26.173504
1100000 2546.745344 2021-10-04 02:15:48.770325
1102000 2547.269632 2021-10-04 02:16:11.417554
1104000 2548.056064 2021-10-04 02:16:33.600896
1106000 2548.580352 2021-10-04 02:16:56.042438
1108000 2562.609152 2021-10-04 02:17:20.576953
1110000 2563.13344 2021-10-04 02:17:44.333912
1112000 2563.919872 2021-10-04 02:18:07.679960
1114000 2564.44416 2021-10-04 02:18:30.421250
1116000 2565.230592 2021-10-04 02:18:53.263856
1118000 2597.138432 2021-10-04 02:19:17.496385
1120000 2597.924864 2021-10-04 02:19:40.669181
1122000 2598.449152 2021-10-04 02:20:03.664045
1124000 2599.231488 2021-10-04 02:20:26.427597
1126000 2599.75

,hylak_id,final_g,final_v,source,default
0,18,"[1, 2, 3, 4, 5, 6, 7,...","[475.8309631347656, 4...",srtm_nasa,0
1,157,"[1, 2, 3, 4, 5, 6, 7,...","[1181.1527099609375, ...",alos3_2,0
2,1317,"[1, 2, 3, 4, 5, 6, 7,...","[152.92738037109376, ...",alos3_2,0
3,1319,"[1, 2, 3, 4, 5, 6, 7,...","[86.65804290771484, 8...",alos3_2,0
4,1322,"[1, 2, 3, 4, 5, 6, 7,...","[15.518573379516603, ...",alos3_2,0
...,...,...,...,...,...
1415820,823559,"[9, 10, 11, 12, 13, 1...","[521.56533203125, 521...",srtm_nasa,0
1415821,823992,"[2, 3, 4, 5, 6, 7, 8,...","[485.0, 485.0, 485.13...",srtm_nasa,1
1415822,824108,"[6, 7, 8, 9, 10, 11, ...","[523.8740478515626, 5...",alos3_2,0
1415823,824242,"[1, 2, 3, 4, 5, 6, 7,...","[485.877392578125, 48...",alos3_2,0
